# Lab 4: RNA-seq

Skills: differential expression, basic R, enrichment analysis

For this week you'll need to complete the following, which are described in more detail below:

* `CSE185-LAB4-EXERCISES.ipynb` (20 pts)
* `CSE185-LAB4-REPORT.ipynb` (70 pts) (this notebook)
* `CSE185-LAB4-README.ipynb` (10 pts)

## Intro
You would like to study the effects that eating a high fat diet might have on your health. To explore this, you perform an experiment using mice. You feed three mice a standard "chow" diet and another three mice a high fat diet ("HFD"). After 7 weeks, you collect liver biopsies from each of the six mice and perform RNA-sequencing. You'd like to analyze the RNA-seq results to determine which genes changed their expression, and what those genes might be doing.

Our pipeline will consist of:
1. Alignment of RNA-seq data and quantifying expression levels (we've already done that for you)
2. Differential expression analysis
3. "GO" Enrichment analysis to characterize differentially expressed genes.
4. Visualizing expression data with IGV

Note, the data from this lab is from the paper [High fat diet-induced changes of mouse hepatic transcription and enhancer activity can be reversed by subsequent weight loss *Scientific Reports* 2017](https://www.nature.com/articles/srep40220.pdf).

### Note on new lab notebook format
This week, we'll be transitioning to a new lab format which is no longer autograded and therefore a bit more open-ended. You'll be filling out the following:

* `CSE185-LAB4-REPORT.ipynb`: will provide instructions (and hints) about the analyses you'll be performing that week. It will have prompts for the questions you should answer. <font color="red">This file is graded. The report is worth 70 pts (70%) of the points for this week's lab.</font> From here on out, we will not be asking you to paste specific commands you used to get your answer in the actual report. Instead, we will ask you to document your commands in the following file:

* `CSE185-LAB4-README.ipynb`: We have included a mostly blank document for you. You should use this file to keep track of exactly the commands you used for each analysis, so that you can come back to it next week (or next year!) and remember what you did. <font color="red">This file is graded. Documentation is worth 10 pts (10%) of the points for this week's lab.</font>

There are additional exercises in `CSE185-LAB4-EXERCISES.ipynb`. <font color="red"> As usual exercises are worth a total of 20 pts (20%) for this week's lab.</font> You'll also see `CSE185-LAB4-PREPROCESSING.ipynb` with background information for how we performed the alignment and expression quantification steps.

### Summary of tools covered
In this lab we'll be using the following tools and resources:

* [Gene Expression Omnibus](https://www.ncbi.nlm.nih.gov/geo) to access publicly available data.
* [IGV](https://software.broadinstitute.org/software/igv/) a genome browser. Used for visualizing RNA-seq alignments (and other types of genomic data)
* [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) for differential expression analysis. This is an R package. It takes in gene expression levels across multiple replicates of multiple conditions and determines which genes are differentially expressed.
* [PANTHER](http://www.pantherdb.org/) (or your tool of choice) for Gene Ontology analysis.

We will also refer to these tools but will not run them directly:
* [STAR](https://github.com/alexdobin/STAR) for aligning RNA-seq reads to a reference transcriptome.
* [RSEM](https://deweylab.github.io/RSEM/) for quantifying gene expression. RSEM takes aligned reads as input and outputs expression levels for each gene.

Note: Because DESeq2 is an R package, we'll be writing a small bit of R code. We'll point you to some resources to help you figure this out.

And as usual, we'll do some plotting with the [matplotlib](https://matplotlib.org/) Python library (or, whatever method you want to use for plotting. You do not have to use matplotlib. You can also write R code directly in Jupyter notebooks as we'll see below).

### Summary of data provided

For this lab, to save you time we have already precomputed part of the results you will need to complete the lab, which can be found in `~/public/lab4/`. You can read about the steps we used for this in `CSE185-LAB4-PREPROCESSING.ipynb`. We will go over how data was obtained from the paper and preprocessed in lecture.

You should see:
* `*.genes.results`: gene-level expression results output by RSEM for each condition/replicate.
* `GRCm38.75.gene_names` contains a mapping between ENSEMBL gene ids (`ENSMUSG...`) and gene names (e.g. Cdc45, Klf6)

## 1. Quantifying gene expression

We have already aligned raw reads to the reference transcriptome using STAR and quantified expression of each gene using RSEM. You can find the results in `~/public/lab4/*.genes.results`.

Take a look at the output file. You will see (among other things): gene_id (a funny looking string like ENSMUSG...), a list of transcript ids for that gene, the length of the gene, and the estimated expression level given in both TPM and FPKM. Refer to the exercises and the lecture slides for more info about the difference between these metrics.

<font color="red">**Question 1 (5 pts)**</font> Summarize the expression results by reporting how many genes were expressed (TPM>0) in each sample. Report your results in a table with columns "Dataset" (e.g. Chow_Rep1, Chow_Rep2, etc.) and "# genes". You should have a total of 6 rows (3 each for Chow and HFD). ([helpful Markdown table syntax](https://www.markdownguide.org/extended-syntax/#tables))

| Dataset      | # Genes |
| ----------- | ----------- |
| Chow_Rep1   | 16918       |
| Chow_Rep2   | 17095        |
| Chow_Rep3   | 16794       |
| HFD_Rep1    | 17077        |
| HFD_Rep2    | 16846       |
| HFD_Rep3    | 16733        |

<font color="red">**Question 2 (10 pts)**</font> Since we included three replicate samples for each condition, one way we can assess the quality of our results is by comparing replicate samples. Provide scatter plots comparing TPM values in Chow Rep1 vs. Rep2 and HFD Rep1 vs. Rep2. Since expression values have a large range, you should compare log10 TPM values rather than raw values themselves. Make sure to label the axes on your plots. Provide some interpretation of your plots - do the expression values between replicates seem to be well correlated or not?

Note: you can insert figures by saving them to a file and including them here using [markdown syntax](https://medium.com/markdown-monster-blog/getting-images-into-markdown-documents-and-weblog-posts-with-markdown-monster-9ec6f353d8ec).

**Chow Comparison**
![](chowchart.png)

**HFD Comparison**
![](hfdchart.png)

## 2. Differential expression analysis

Now that we have quantified gene expression in each of our samples, we can perform differential expression analysis. For this, we will be using DESeq2, a widely used method that takes in estimated gene expression levels across different samples and outputs a set of differentially expressed genes.

This tool is an R package, and so must be run from R. This means we have to learn at least a bit of a new language. Fortunately, we can write R code directly in Jupyter notebooks using the R "cell magic" (see below). Alternatively, you can open a separate "R" document from JupyterHub or use R at the terminal if that's more comfortable. 

The example cells below shows how you can turn a cell into an R cell. It also loads `DESeq2` and `tximport`, two libraries you will need, and provides some helpful comments to get you started.

**Use DESeq2 to identify genes that are differentially expressed between Chow and HFD**. You'll need to write some R code that does the following:

* Set up the list of files to be used as input to DESeq2. We will input the RSEM results from each of our replicates in each condition. Recall these can be found at `~/public/lab4/*.genes.results`.
* Import the data to DESeq2 using the tximport library.
* Run DESeq2 to find differentially expressed genes between the two conditions.
* Output results to a file `chow_vs_hfd_deseq2.csv`.

We recommend taking a look at the following tutorial: http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html. We'll also be going over some hints in lab sessions.

In [3]:
# Run this to allow using the %%R cell magic
%load_ext rpy2.ipython

# Suppress warnings (most notable from rpy2)
import warnings
warnings.filterwarnings('ignore')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [6]:
%%R

##### Load the libraries we need #####
library("DESeq2")
library("tximport")

# This cell is an example. We recommend putting the R code you use for this
# in the CSE185-LAB4-README.ipynb notebook

# You might find some of the code below helpful!
# Or, you can ignore what we have below and follow
# http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html

##### List the files and set up metadata #####
# Note, you should change this to use the files in your home directory
#files <- c("Chow_Rep1.genes.results",
#         "Chow_Rep2.genes.results",
#         "Chow_Rep3.genes.results",
#         "HFD_Rep1.genes.results",
#         "HFD_Rep2.genes.results",
#         "HFD_Rep3.genes.results")
#conditions <- c(rep("Chow", 3), rep("HFD", 3))
#samples <- data.frame("run"=c("Chow_Rep1", "Chow_Rep2", "Chow_Rep3", "HFD_Rep1", "HFD_Rep2", "HFD_Rep3"),
#                    "condition"=conditions)
#names(files) = samples$run

##### Use "tximport" to convert RSEM results to the format needed by DESeq2 #####
# TODO
# 1. Import RSEM results with tximport
# 2. Note, we also used txi$length[txi$length == 0] <- 1 
# to add a pseudocount of 1 to fix an error with 0-length transcripts
# 3. Load to a deseq dataset from tximport (see DESeqDataSetFromTximport)

###### Filter things with very low counts so we don't waste time on those #####
# TODO

##### Perform deseq2 #####
# TODO

##### Write results to chow_vs_hfd_deseq2.csv #####
# TODO

After DESeq2 runs successfully, you should get a csv file with multiple columns:
* Gene id (e.g. ENSMUSG00000000088)
* log2FoldChange: gives the log2 of the fold change in expression of the gene between conditions
* pvalue: gives the nominal p-value for each gene, where here the null hypothesis is that the gene is expressed equally in Chow vs. HFD.
* padj: gives the pvalues "adjusted" for the number of hypotheses being tested (false discovery rate).

For downstream analyses, **we recommend treating genes with adjusted pvalue $<$ 0.05 as significant**.

<font color="red">**Question 3 (5 pts)**</font> Describe how you performed differential expression analysis. Which package(s) did you use? Which version(s)? Did you have to do any preprocessing of your data (e.g. removing genes with length 0 or removing genes with very low counts)? Justify any choices you made about which genes to filter or any parameters you had to set.

I performed this differential expression analysis using the DESeq2 package in R that was specificed in the lab 4 hints as well as above. The transcript-level abundance files were imported using tximport which was also a package that I imported following the guidelines that were provided. I also did the addition of a pseudocount to mitigate issues associated with genes having zero counts. Next, genes with low expression levels were filtered out to enhance the accuracy of the analysis, I chose a threshold of 8, somewhat arbitrarily however, I thought any number between the 5-10 range would be resonable as otherwise the expression levels are too low to get any credible results from their analysis (however, having been told we expected around 15000) I assumed I was on the right track. After all this, differential expression analysis was performed using the DESeq function, and the results were obtained using the results function. Both non-significant and significant genes, determined by an adjusted p-value (the adjusted score) cutoff of 0.05, were written to separate CSV files. All of these choices, including adding pseudocounts, filtering low-expressed genes, and selecting a good p-value threshold, were made to ensure robust and reliable identification of differentially expressed genes while controlling for false discoveries
.

<font color="red">**Question 4 (15 pts)**</font> Visualize the differential expression results using a "volcano plot", which plots the log2 fold change vs. the -log10 p-value. Make your volcano plot more informative by: (1) coloring genes that are significantly differentially expressed in a different color and (2) annotating the names of the top differentially expressed genes. 

**Volcano Plot!**
![](volcano_plot.png)


<font color="red">**Question 5 (5 pts)**</font> At an adjusted p-value threshold (FDR) of 5%, how many genes were differentially expressed across the two conditions? Report the top 10 most differentially expressed genes, their log2 fold change, and the p-value. Report the gene names (e.g. Fads2) rather than gene ids.

You'll notice all the gene ids are pretty cryptic looking (e.g. ENSMUSG00000000088). We have provided a file, `GRCm38.75.gene_names` in the `~/public/lab4` directory that can be used to convert these to gene names.

**Answer:** There were 375 genes that were differentially expressed. Below is a table of the important values including names of the top 10 genes. The first tables is just considering the log2fold Change. The second table considers the padj values.  

|       Ensemble ID          | gene_name |      padj      |     pvalue     | log2FoldChange |
|:-----------------:|:---------:|:--------------:|:--------------:|:--------------:|
| ENSMUSG00000096617 |   Gm5559  | 2.582164e-05   | 1.400062e-07   |     9.684375   |
| ENSMUSG00000083678 |  Gm12989  | 8.450639e-04   | 8.711416e-06   |     6.516816   |
| ENSMUSG00000060096 |  Amd-ps3  | 3.436874e-03   | 4.570638e-05   |    -6.301650   |
| ENSMUSG00000050505 |   Pcdh20  | 9.041397e-03   | 1.423761e-04   |     6.126644   |
| ENSMUSG00000032739 |   Pram1   | 9.041397e-03   | 1.417694e-04   |    -5.892079   |
| ENSMUSG00000020000 |   Moxd1   | 1.055707e-04   | 7.349458e-07   |     5.586066   |
| ENSMUSG00000059430 |   Actg2   | 4.229139e-02   | 1.016307e-03   |     5.375245   |
| ENSMUSG00000001349 |   Cnn1    | 1.316395e-02   | 2.255822e-04   |     5.307799   |
| ENSMUSG00000028715 |  Cyp4a14  | 9.273004e-07   | 3.972637e-09   |    -5.153536   |
| ENSMUSG00000030483 |  Cyp2b10  | 7.291275e-37   | 4.880698e-41   |    -5.083903   |

**Based on P-adjusted Value**

| Gene ID            | Gene Name         | padj            | pvalue          | log2FoldChange |
|:--------------------:|:-------------------:|:-----------------:|-----------------:|:----------------:|
| ENSMUSG00000030483 | Cyp2b10           | 7.291275e-37    | 4.880698e-41    | -5.083903      |
| ENSMUSG00000024665 | Fads2             | 5.837360e-31    | 7.814927e-35    |  1.545783      |
| ENSMUSG00000003053 | Cyp2c29           | 1.480674e-30    | 2.973440e-34    | -1.835254      |
| ENSMUSG00000055730 | Ces2a             | 3.963661e-30    | 1.061292e-33    | -1.600664      |
| ENSMUSG00000059824 | Dbp               | 7.488471e-26    | 2.506349e-29    |  2.637499      |
| ENSMUSG00000057933 | Gsta2             | 1.297864e-24    | 5.212655e-28    | -2.836965      |
| ENSMUSG00000058135 | Gstm1             | 1.995408e-24    | 9.349928e-28    | -1.360462      |
| ENSMUSG00000020865 | Abcc3             | 4.757728e-21    | 2.547816e-24    | -1.457775      |
| ENSMUSG00000067225 | Cyp2c54           | 6.231439e-18    | 3.754130e-21    | -1.732680      |
| ENSMUSG00000032080 | Apoa4             | 1.735208e-17    | 1.161529e-20    |  2.544927      |



## 3. Gene ontology enrichment analysis

Finally, use gene ontology (GO) enrichment analysis to characterize the genes that were differentially expressed in Chow vs. HFD. We recommend you use [PANTHER](http://www.pantherdb.org/) or [DAVID](https://david.ncifcrf.gov/tools.jsp), which are web tools, for doing this. However you are welcome to try out other GO analysis tools instead. You can find a helpful tutorial on DAVID here: https://david.ncifcrf.gov/helps/tutorial.pdf. 

For enrichment analysis, you will need to define our "foreground" and "background". Here:

* "Background" consists of all genes that we analyzed (i.e., all genes with any output in the DESeq2 csv file).
* "Foreground" consists of all genes that were differentially expressed (padj<0.05). You should create separate foreground sets for up- and down-regulated genes. Recall up-regulated genes will have log2 fold change > 0, and down-regulated genes will have log2 fold change < 0.

You can upload lists of genes to use as foreground or background. In our solution, we uploaded lists of the gene IDs (ENSG...) to DAVID. Then DAVID will perform GO enrichment analysis for you.

**Perform GO enrichment by comparing your differentially expressed genes (adjusted p<0.05) to all genes analyzed. You should perform a separate analaysis for down-regulated (log2 fold change <0) and up-regulated (log2fold change>0) genes.**

<font color="red">**Question 6 (10 pts)**</font> Describe any methods for how you performed gene ontology enrichment analysis. Which tool did you use? How did you define your foreground and background? Were there any extra options you had to set? Justify what you used for each of those.

I used DAVID to perform GO enrichment analysis. I defined foreground as either my upregulated or downregulated genes adn then my background as my full set of genes. In regards to where I looked, I did not take into account the 'ALL' segments of the specific genes because I wanted to look at the DIRECT, as it did not take into consider the children in the ontology/ similar words/processes. This is also because I wanted to keep it consistent when looking at the upregulated and downregulated processes so I just chose to stick with the direct as opposed to all. 

<font color="red">**Question 7 (5 pts)**</font> Were any gene ontology categories enriched in differentially expressed genes? For each enriched category, provide at least the name of the category and its enrichment p-value.

There are three major categories that pop up when looking at the enrichmenet analysis: BP (Biological Processes), CC (cellular components) and MF (molecular function). Overall, the most up and down regulated were in the category of BP and for downregulated specifically there were some special cases in MF with especially low P values. Further, I just looked at DIRECT of each as I did not want to consider all the children/ related terms. 

**Upregulated:**

| GO Term                        | Enrichment P-Value | Category  |
|--------------------------------|--------------------|-----------|
| Cholesterol biosynthetic process | 1.9E-17            | BP DIRECT |
| Lipid metabolic process          | 8.3E-16            | BP DIRECT |
| Sterol biosynthetic process      | 9.5E-16            | BP DIRECT |
| Cholesterol metabolic process    | 3.4E-15            | BP DIRECT |
| Steroid biosynthetic process     | 5.6E-13            | BP DIRECT |


**DownRegulated**

| GO Term                                                             | Enrichment P-Value | Category   |
|---------------------------------------------------------------------|--------------------|------------|
| Arachidonic acid metabolic process                                 | 2.0E-16            | BP DIRECT  |
| Xenobiotic metabolic process                                       | 1.3E-11            | BP DIRECT  |
| Linoleic acid metabolic process                                    | 7.3E-11            | BP DIRECT  |
| Glutathione metabolic process                                      | 2.7E-10            | BP DIRECT  |
| Iron ion binding                                                    | 2.2E-15            | GOTERM MF DIRECT |
| Oxidoreductase activity, acting on paired donors, with incorporation or reduction of molecular oxygen | 3.4E-15  | MF DIRECT  |
| Monooxygenase activity                                              | 1.2E-14            | MF DIRECT  |
| Arachidonic acid epoxygenase activity                               | 2.1E-14            | MF DIRECT  |



## 4. Discussion questions

Answer the following discussion questions below:

<font color="red">**Question 8 (5 pts)**</font> Summarize the characteristics of genes that are up- and down-regulated in HFD compared to Chow fed mice. Do the GO categories enriched in those gene sets make sense? Do a little exploring of the top differentially expressed genes to find out what they might be doing.

All the key characteristics of the up-regulated and down-regulated make sense with high fat diet. Specifically, upregulated genes mainly cluster around biological processes associated with cholesterol and lipid metabolism, like cholesterol biosynthetic processes, lipid metabolic processes, and sterol biosynthesis. These findings align with expectations, this is because high-fat diets often lead to increased lipid intake and subsequent metabolic demands. The upregulation of genes involved in cholesterol and lipid metabolism shows an adaptive response to the elevated dietary fat intake, suggesting an increased need for lipid processing and storage. At the same time, downregulated genes are usually enriched in biological processes related to metabolic pathways such as arachidonic acid, xenobiotics, linoleic acid, and glutathione metabolism. This downregulation may show a reduction in inflammatory and detoxification processes. For example, the downregulation of genes involved in arachidonic acid metabolism suggests a dampened inflammatory response, while decreased xenobiotic metabolism gene expression may reduce the body's ability to metabolize and eliminate foreign substances. These findings show that there is a omplex interplay between dietary factors and metabolic regulation, where a high-fat diet may modulate gene expression to favor lipid metabolism while potentially compromising inflammatory and detoxification pathways.

<font color="red">**Question 9 (5 pts)**</font> You'll likely notice from your volcano plot that there are many genes with very high fold changes (e.g. log2 fold change > 6) but that are not called as significant. Hypothesize why. it could be helpful to dig in and look at the raw data (e.g. expression levels from RSEM) for some of those genes.

There are genes with high fold changes but that are not called significant ebcasue genes with low counts tend to have higher variability in their expression levels. This makes it harder to distinguish true biological signals from random fluctuations or noise in the data. Even if they have a large fold changes, the high variability associated with lowly expressed genes may mean that we have a wider confidence interval and lower statistical significance, which then leads to non-significant p-values. Examining the raw expression data, such as expression levels from RSEM, for these genes could provide insights into the underlying variability and help elucidate whether the observed changes are biologically meaningful or simply due to noise. Additionally, considering alternative statistical methods or conducting functional validation experiments may further clarify the significance of these genes in the context of the biological system under investigation.

<font color="red">**Question 10 (5 pts)**</font> Multiple factors contribute to our statistical power to detect differentially expressed genes. (Power is the probability that we reject the null hypothesis, given that there is really an effect). How might our power be affected by the number of replicates? (here we had 3 replicates each). What about gene length?

Increasing the number of replicates makes the precision of our estimates better and also reduces the impact of random fluctuations or noise in the data. With more replicates, we can better estimate the true underlying distribution of gene expression levels and improve our ability to detect subtle changes in expression associated with experimental conditions. gene length indirectly affects statistical power through its impact on read counts and expression estimates. Regarding the second part of the question about gene length, longer genes typically produce more RNA transcripts. The means there is a higher read counts and increased statistical power to detect differential expression. This is because higher read counts provide more reliable estimates of gene expression levels and also make our ability to detect significant differences between experimental conditions alot better. However, there are naunces to gene length and power because longer genes may also have higher variability in expression due to their complexity and regulatory elements, which can reduce statistical power. Additionally, longer genes may require higher sequencing depth to accurately capture their expression profiles, which can increase experimental costs and computational burden.

## Optional: Visualizing alignments in IGV

It is a good practice to visualize our alignments, both to see that the alignment worked well, and also to see whether things we report as differentially expressed make sense. We'll use a genome browser called the Integrative Genomics Viewer, or IGV. In the process, we'll also see how to access data published with the original paper.

Navigate to the web version of IGV: https://igv.org/app/ (or alternatively, download the desktop version which we actually prefer to use). Set the genome to (mm9), since according to the paper they had aligned reads to the mm9 reference genome (see page 11, "Analysis of RNA, ChIP and DNase sequencing"). Note, in our lab we aligned the data to the newer mm10/GRCm38 reference.

Take a moment to orient yourself with IGV. It is basically like a Google Maps for genomes! The top gives the names of each chromosome. The bottom track, labeled "Refseq genes" gives the names and coordinates for all annotated genes. Let's choose one to look at. Type "Fads2" in the search box at the top. This will zoom the view in on this gene. Notice how in the gene you can see the exon and intron structure. The little arrows in the introns point to the left, which means this gene is on the reverse strand of the reference. Take a look at another gene (e.g. Fads3) to see a gene on the forward strand. Zoom in further (using the "+" at the top right) until you can see actual DNA sequence at the top.

Now we'd like to load our sequence alignments. While IGV can directly visualize BAM files, we'll instead look at "counts" files (".bedgraph" format. see http://genome.ucsc.edu/goldenPath/help/bedgraph.html) which are much smaller. These give read counts per position (i.e. coverage) which can give us an idea of the abundance of each gene. 

The authors of the paper have already provided data in bedgraph format. Head to the paper where we got the data: [High fat diet-induced changes of mouse hepatic transcription and enhancer activity can be reversed by subsequent weight loss *Scientific Reports* 2017](https://www.nature.com/articles/srep40220.pdf). On page 12, you'll see a section "Additional Information" which contains information about where to access the raw data. If a paper generates new sequencing or other data types, they are usually required to make this information available. It will often be in a section labeled "Data availability" or something similar. We used the polyA RNA-seq, which you should find has been deposited in the Gene Expression Omnibus under accession GSE87565.

Go to the [Gene Expression Omnibus](https://www.ncbi.nlm.nih.gov/geo/) and search using the accession GSE87565. Scroll all the way to the bottom to find a section labeled "Supplementary file". If you click "custom" under the "Download" column of that table, you'll be able to select individual datasets to download. Check the boxes for:

* GSM2333833_SM1959_RNA_Chow_rep1.bedgraph.gz
* GSM2333834_SM1960_RNA_Chow_rep2.bedgraph.gz
* GSM2333835_SM1961_RNA_Chow_rep3.bedgraph.gz
* GSM2333839_SM1965_RNA_HFD_rep1.bedgraph.gz
* GSM2333840_SM1966_RNA_HFD_rep2.bedgraph.gz
* GSM2333841_SM1967_RNA_HFD_rep3.bedgraph.gz

After checking those, click "Download" to download to your local computer. Once the downloaded is complete, decompress the .tar file (it will be named something like `GSE87565_RAW.tar`.

Now, go back to IGV in your web browser. Click "Tracks" -> "Local File", then navigate to the bedgraph.gz files you had downloaded in order to upload them to IGV.

Navigate again to Fads2 (This should be one of your differentially expressed genes!). Note that the RNA-seq tracks have very "spiky" coverage. Some regions have tons of reads and others are flat. Note how that compares to the structure of the gene annotated on the bottom. As expected, the "spikes" correspond to reads from exons, since intron and intergenic sequences generally aren't sequenced in our RNA-seq experiment.

Take note of the y-axis range for each dataset (after you initially load them, it should range from 0 to a couple hundred for this gene). To make the tracks more comparable, you can set them all to have the same data range by using the "settings" icon to the left of each track and clicking "set data range". Try setting them all to 500. (the appropriate range will be different for different genes, since genes are really highly expressed whereas others are very lowly expressed. See the huge range on your scatterplot!) You should see that the gene is much more highly expressed in the HFD samples.

It is also helpful to color tracks. For instance, you can color tracks by condition (see options in settings to the right of each track). Navigate to different differentially expressed genes and see if the data matches up with your results from DESeq2.

<font color="red">Note: the bedgraph files from GEO look a little weird (the coverage peaks are too wide), which may be the result of an extension step being applied. We also provided tdfs ([here](https://drive.google.com/drive/folders/1L0uu5QSVgU3LSxybX50bDhGuv_vU2vTH?usp=sharing)) that we created from the aligned BAM files after we ran STAR. You can also visualize these using IGV, just be sure to change to the mm10 genome. Visualizations in the slides are based on these tdfs.</font>

We will continue using IGV to explore ChIP-seq datasets next week.